Reference Links for Corner Detection, Optical Flow

https://www.geeksforgeeks.org/python-corner-detection-with-shi-tomasi-corner-detection-method-using-opencv/
https://docs.opencv.org/3.4/dc/d6b/group__video__track.html
https://nanonets.com/blog/optical-flow/

In [2]:
import numpy as np
import cv2 

In [3]:
# Parameters for ShiTomasi corner detection (good features to track paper)


# Shi Tomasi does not detect the corners like Harris Method. We ought to flat out the array 
# and draw the circles that indicates the corners.

# For example, if the best corner has the 
# quality measure = 1500, and the qualityLevel=0.01 , then all the corners with the quality measure
# less than 15 are rejected.

corner_track_params = dict(maxCorners = 10,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

In [4]:
# Parameters for Lucas Kanade optical flow
# Very Small window more sensitive, missing larger motions
# maxlevel --> image pyramid  - 0 -(original image -1.0 resolution) , 1 - (0.5 resolution), etc..

# OPENCV Documentation________________________________
# COUNT 	 the maximum number of iterations or elements to compute
# EPS 	 the desired accuracy or change in parameters at which the iterative algorithm stops

lk_params = dict( winSize  = (200,200),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,0.03))

In [4]:
# Capture the video
cap = cv2.VideoCapture(0)

# Capture the very first frame of the stream
ret, prev_frame = cap.read()

# Grab a grayscale image 

# We will refer to this as the previous frame

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Grabbing the corners, these are the points we will be tracking
# While passing in a dict into a function, we do it by ** in front of it

prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params) 

# Create a matching mask of the previous frame for drawing on later
# Creates a np array of zeroes same size of prev_fame

mask = np.zeros_like(prev_frame)


while True:
    
    # Grab current frame
    ret,frame = cap.read()
    
    # Grab gray scale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate the Optical Flow on the Gray Scale Frame
    
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    
    # Using the returned status array (the status output)
    # status output status vector (of unsigned chars); each element of the vector is set to 1 if
    # the flow for the corresponding features has been found, otherwise, it is set to 0.
    
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    # Use ravel to get points to draw lines and circles
    
    # Remember we have got 10 pts to track___and for every pt in the set of 10 we use the loop below to 1) draw a line between 
    # the prev and current points 2) We draw a new circle at the the 10 new points positions
    
    
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        
        #ravel -----> np method to make an array into 1-d array (flattening it)
        
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
        # Lines will be drawn using the mask created from the first frame
        mask = cv2.line(mask, (x_new,y_new),(x_prev,y_prev), (0,255,0), 3)
        
        # Draw red circles at corner points
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
    
    # Display the image along with the mask we drew the line on.
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()
cap.release()